In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import torchaudio
import yaml
import pickle

from collections import namedtuple
from pathlib import Path
from snac import SNAC
from train_tokenizer import VQVAEModule
import imageio
from src.dataset import Dataset
from src.tokenizer_module import TokenizerModule
from utils.tokenizer_utils import prepare_features, load_fsq_vae, repackage_output
import matplotlib.pyplot as plt

In [3]:

# model = SNAC.from_pretrained("hubertsiuzdak/snac_32khz").eval().cuda()
# audio = torch.randn(1, 1, 32000).cuda()  # placeholder for actual audio with shape (B, 1, T)

In [4]:
# # load audio from .wav file
# # Load and preprocess audio
# waveform, sample_rate = torchaudio.load("dataset/audio/_a-pB_5eRt0_7.wav")
# # Convert to mono if stereo
# if waveform.shape[0] > 1:
#     waveform = torch.mean(waveform, dim=0, keepdim=True)
# # Resample to 32kHz if needed
# if sample_rate != 24000:
#     resampler = torchaudio.transforms.Resample(sample_rate, 24000)
#     waveform = resampler(waveform)
# # Add batch dimension and move to GPU
# audio = waveform.unsqueeze(0).cuda()

# with torch.inference_mode():
#     codes = model.encode(audio)
#     audio_hat = model.decode(codes)

In [ ]:
rest_fsq, rest_feats = load_fsq_vae(Path("outputs/modified_rest_fsq_D4/checkpoints/checkpoint_epoch=319.ckpt"))

In [ ]:
ds = Dataset("dataset", split="eval", val_split=0.1, seed=2, compute_stats=False)

In [ ]:
sample = ds[134] 

pickle_path = sample['metadata']['pickle_path']
vid_id = pickle_path.split("/")[-1].split(".")[0]
vid_path = f"dataset/train/{vid_id}.mp4"

# Read the first frame from the video and display it
frame = imageio.get_reader(vid_path).get_data(0)
plt.imshow(frame)
plt.show()

In [ ]:
# tokenizer = TokenizerModule(tokenizer_config="configs/tokenizer_config.yaml")
tokenizer = TokenizerModule.from_pretrained("InternalCan/tokenizer_module")

In [ ]:
sample['metadata']

In [10]:
indices = tokenizer.encode(sample)

In [ ]:
reconstr_feats = tokenizer.decode(indices)

In [12]:
new_dims = {
    'R': 9,
    'c_eyes_lst': 2,
    'c_lip_lst': 1,
    'exp': 63,
    'kp': 63,
    'scale': 1,
    't': 3,
    'x_s': 63
}

In [28]:
new_rest_feats = {}

for key, value in rest_feats.items():
    new_rest_feats[key] = value
    new_rest_feats[key]['enabled'] = True

In [ ]:
output = repackage_output(sample, reconstr_feats, ds, dims=new_dims, feats_data=new_rest_feats)

In [22]:
# # Replace reconstructed features with original ones for a specific range
# feat = 'exp'

# feat_range = (0, 21)

# denormalized_sample = ds.denormalize_features(sample[feat], feat)

# for i in range(output['n_frames']):
#     output['motion'][i][feat][:, feat_range[0]:feat_range[1]] = 0

In [ ]:
pickle_dir = Path("dataset/pickles")
video_id = Path(pickle_path).stem
new_path = pickle_dir / f"{video_id}_reconstructed.pkl"
print(new_path)
with open(new_path, "wb") as f:
    pickle.dump(output, f)

In [ ]:
!python inference.py -d {new_path} -s assets/examples/source/reconstructed.png

In [ ]:
# tokenizer.save_pretrained("tokenizer_module")
# tokenizer.push_to_hub("InternalCan/tokenizer_module")